# MIND 数据集重新划分

## 目的
原始 test 数据缺少点击标签,无法用于训练。本脚本将 train 和 val 数据合并后,按照 7:2:1 比例重新划分为训练集、验证集和测试集。

## 策略
- 按用户随机划分,避免数据泄漏
- 使用固定 random_seed=42 保证可复现性
- 同一用户的所有行为记录分配到同一数据集

In [2]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

## 阶段 1: 合并 news.tsv

In [3]:
# 读取 train 和 val 的 news.tsv
print("正在读取 news.tsv...")
news_train = pd.read_csv('./data/MIND/train/news.tsv', sep='\t', header=None)
news_val = pd.read_csv('./data/MIND/val/news.tsv', sep='\t', header=None)

print(f"train news 数量: {len(news_train)}")
print(f"val news 数量: {len(news_val)}")

# 合并并去重
news_merged = pd.concat([news_train, news_val], ignore_index=True)
news_merged = news_merged.drop_duplicates(subset=[0], keep='first')

print(f"合并后的 news 数量: {len(news_merged)} (去重后)")

# 创建输出目录
os.makedirs('./data/MIND/merged', exist_ok=True)

# 保存合并后的 news.tsv
news_merged.to_csv('./data/MIND/merged/news.tsv', sep='\t', header=False, index=False)
print("已保存: ./data/MIND/merged/news.tsv")

正在读取 news.tsv...
train news 数量: 101527
val news 数量: 72023
合并后的 news 数量: 104151 (去重后)
已保存: ./data/MIND/merged/news.tsv


## 阶段 2: 合并 behaviors.tsv

In [4]:
# 读取 train 和 val 的 behaviors.tsv
print("正在读取 behaviors.tsv...")
behaviors_train = pd.read_csv('./data/MIND/train/behaviors.tsv', sep='\t', header=None)
behaviors_val = pd.read_csv('./data/MIND/val/behaviors.tsv', sep='\t', header=None)

# 处理 NaN 值
behaviors_train = behaviors_train.fillna('')
behaviors_val = behaviors_val.fillna('')

print(f"train behaviors 数量: {len(behaviors_train):,}")
print(f"val behaviors 数量: {len(behaviors_val):,}")

# 合并
behaviors_merged = pd.concat([behaviors_train, behaviors_val], ignore_index=True)

# 重新生成曝光ID (从1开始)
behaviors_merged[0] = range(1, len(behaviors_merged) + 1)

print(f"合并后的 behaviors 数量: {len(behaviors_merged):,}")

# 保存合并后的 behaviors.tsv
behaviors_merged.to_csv('./data/MIND/merged/behaviors.tsv', sep='\t', header=False, index=False)
print("已保存: ./data/MIND/merged/behaviors.tsv")

正在读取 behaviors.tsv...
train behaviors 数量: 2,232,748
val behaviors 数量: 376,471
合并后的 behaviors 数量: 2,609,219
已保存: ./data/MIND/merged/behaviors.tsv


## 阶段 3: 按用户划分数据集 (7:2:1)

In [5]:
# 设置随机种子
np.random.seed(42)

# 获取唯一用户列表
unique_users = behaviors_merged[1].unique()  # 列1是用户ID
print(f"唯一用户数: {len(unique_users):,}")

# 随机打乱用户
np.random.shuffle(unique_users)

# 按 7:2:1 划分用户
n_users = len(unique_users)
train_idx = int(n_users * 0.7)
val_idx = int(n_users * 0.9)

train_users = set(unique_users[:train_idx])
val_users = set(unique_users[train_idx:val_idx])
test_users = set(unique_users[val_idx:])

print(f"\n用户划分:")
print(f"训练集用户: {len(train_users):,} ({len(train_users)/n_users*100:.1f}%)")
print(f"验证集用户: {len(val_users):,} ({len(val_users)/n_users*100:.1f}%)")
print(f"测试集用户: {len(test_users):,} ({len(test_users)/n_users*100:.1f}%)")

# 验证用户无交集
assert len(train_users & val_users) == 0, "训练集和验证集用户有交集!"
assert len(train_users & test_users) == 0, "训练集和测试集用户有交集!"
assert len(val_users & test_users) == 0, "验证集和测试集用户有交集!"
print("\n✓ 用户无交集验证通过")

唯一用户数: 750,434

用户划分:
训练集用户: 525,303 (70.0%)
验证集用户: 150,087 (20.0%)
测试集用户: 75,044 (10.0%)

✓ 用户无交集验证通过


In [6]:
# 根据用户分配行为记录
print("\n正在分配行为记录...")
behaviors_new_train = behaviors_merged[behaviors_merged[1].isin(train_users)].copy()
behaviors_new_val = behaviors_merged[behaviors_merged[1].isin(val_users)].copy()
behaviors_new_test = behaviors_merged[behaviors_merged[1].isin(test_users)].copy()

# 重新生成曝光ID
behaviors_new_train[0] = range(1, len(behaviors_new_train) + 1)
behaviors_new_val[0] = range(1, len(behaviors_new_val) + 1)
behaviors_new_test[0] = range(1, len(behaviors_new_test) + 1)

print(f"\n行为记录划分:")
print(f"训练集: {len(behaviors_new_train):,} 条 ({len(behaviors_new_train)/len(behaviors_merged)*100:.1f}%)")
print(f"验证集: {len(behaviors_new_val):,} 条 ({len(behaviors_new_val)/len(behaviors_merged)*100:.1f}%)")
print(f"测试集: {len(behaviors_new_test):,} 条 ({len(behaviors_new_test)/len(behaviors_merged)*100:.1f}%)")
print(f"总计: {len(behaviors_new_train) + len(behaviors_new_val) + len(behaviors_new_test):,} 条")


正在分配行为记录...

行为记录划分:
训练集: 1,823,973 条 (69.9%)
验证集: 523,103 条 (20.0%)
测试集: 262,143 条 (10.0%)
总计: 2,609,219 条


## 阶段 4: 保存划分后的数据

In [7]:
# 创建输出目录
os.makedirs('./data/MIND/new_split/train', exist_ok=True)
os.makedirs('./data/MIND/new_split/val', exist_ok=True)
os.makedirs('./data/MIND/new_split/test', exist_ok=True)

# 保存 behaviors.tsv
behaviors_new_train.to_csv('./data/MIND/new_split/train/behaviors.tsv', sep='\t', header=False, index=False)
behaviors_new_val.to_csv('./data/MIND/new_split/val/behaviors.tsv', sep='\t', header=False, index=False)
behaviors_new_test.to_csv('./data/MIND/new_split/test/behaviors.tsv', sep='\t', header=False, index=False)

print("已保存 behaviors.tsv:")
print("  - ./data/MIND/new_split/train/behaviors.tsv")
print("  - ./data/MIND/new_split/val/behaviors.tsv")
print("  - ./data/MIND/new_split/test/behaviors.tsv")

已保存 behaviors.tsv:
  - ./data/MIND/new_split/train/behaviors.tsv
  - ./data/MIND/new_split/val/behaviors.tsv
  - ./data/MIND/new_split/test/behaviors.tsv


In [8]:
# 复制 news.tsv 到三个子目录 (Windows 不支持软链接,使用复制)
import shutil

shutil.copy('./data/MIND/merged/news.tsv', './data/MIND/new_split/train/news.tsv')
shutil.copy('./data/MIND/merged/news.tsv', './data/MIND/new_split/val/news.tsv')
shutil.copy('./data/MIND/merged/news.tsv', './data/MIND/new_split/test/news.tsv')

print("\n已复制 news.tsv 到三个子目录")


已复制 news.tsv 到三个子目录


## 阶段 5: 数据质量验证

In [9]:
print("\n" + "="*60)
print("数据集划分统计报告")
print("="*60)

print(f"\n训练集:")
print(f"  - 行为记录: {len(behaviors_new_train):,} 条")
print(f"  - 用户数: {len(train_users):,} 个")
print(f"  - 平均每用户: {len(behaviors_new_train)/len(train_users):.1f} 条记录")

print(f"\n验证集:")
print(f"  - 行为记录: {len(behaviors_new_val):,} 条")
print(f"  - 用户数: {len(val_users):,} 个")
print(f"  - 平均每用户: {len(behaviors_new_val)/len(val_users):.1f} 条记录")

print(f"\n测试集:")
print(f"  - 行为记录: {len(behaviors_new_test):,} 条")
print(f"  - 用户数: {len(test_users):,} 个")
print(f"  - 平均每用户: {len(behaviors_new_test)/len(test_users):.1f} 条记录")

print(f"\n用户重叠检查:")
print(f"  - train ∩ val: {len(train_users & val_users)} (应为0)")
print(f"  - train ∩ test: {len(train_users & test_users)} (应为0)")
print(f"  - val ∩ test: {len(val_users & test_users)} (应为0)")

print(f"\n新闻库统计:")
print(f"  - 合并后的新闻数: {len(news_merged):,}")

print("\n" + "="*60)
print("✓ 数据重新划分完成!")
print("="*60)
print("\n下一步: 运行 '数据预处理.ipynb' 生成训练样本")


数据集划分统计报告

训练集:
  - 行为记录: 1,823,973 条
  - 用户数: 525,303 个
  - 平均每用户: 3.5 条记录

验证集:
  - 行为记录: 523,103 条
  - 用户数: 150,087 个
  - 平均每用户: 3.5 条记录

测试集:
  - 行为记录: 262,143 条
  - 用户数: 75,044 个
  - 平均每用户: 3.5 条记录

用户重叠检查:
  - train ∩ val: 0 (应为0)
  - train ∩ test: 0 (应为0)
  - val ∩ test: 0 (应为0)

新闻库统计:
  - 合并后的新闻数: 104,151

✓ 数据重新划分完成!

下一步: 运行 '数据预处理_新划分.ipynb' 生成训练样本
